In [1]:
import wandb
import torch
import torch.nn as nn
import numpy as np
from tqdm.auto import tqdm
from torch.utils.data import DataLoader, Dataset, TensorDataset
import torchvision.datasets as ds
import matplotlib.pyplot as plt

In [2]:
def load_cifar(datadir='./data_cache', p=0.0): # will download ~400MB of data into this dir. Change the dir if neccesary. If using paperspace, you can make this /storage
    """This function loads the CIFAR-10 dataset, and applies the random labelling leveraged in the paper Understanding Deep Learning requires rethinking Generalization (2017), 
    Zheng et al. Specifically, we are uniformly accross 10 classes randomizing the labels with probability 1-p"""
    train_ds = ds.CIFAR10(root=datadir, train=True,
                           download=True, transform=None)
    test_ds = ds.CIFAR10(root=datadir, train=False,
                          download=True, transform=None)

    def to_xy(dataset, p):
        X = torch.Tensor(np.transpose(dataset.data, (0, 3, 1, 2))).float() / 255.0  # [0, 1]
        labels = np.array(dataset.targets)
        mask = np.random.uniform(size=labels.shape[0]) < p
        labels[mask] = np.random.randint(0, 10, size=np.sum(mask))
        Y = torch.Tensor(labels).long()
        return X, Y

    X_tr, Y_tr = to_xy(train_ds, p)
    X_te, Y_te = to_xy(test_ds, p)
    return X_tr, Y_tr, X_te, Y_te

def make_loader(dataset, batch_size=128):
    return torch.utils.data.DataLoader(dataset, batch_size=batch_size,
            shuffle=True, num_workers=4, pin_memory=True)

X_tr, Y_tr, X_te, Y_te = load_cifar()
X_tr_noisy, Y_tr_noisy, X_te_noisy, Y_te_noisy = load_cifar(p=0.5)
X_tr_randomized, Y_tr_randomized, X_te_randomized, Y_te_randomized = load_cifar(p=1.0)
train_dl = make_loader(TensorDataset(X_tr, Y_tr), batch_size=64)
train_dl_noisy = make_loader(TensorDataset(X_tr_noisy, Y_tr_noisy), batch_size=64)
train_dl_randomized = make_loader(TensorDataset(X_tr_randomized, Y_tr_randomized), batch_size=64)
train_dls = [train_dl, train_dl_noisy, train_dl_randomized]
test_dl = make_loader(TensorDataset(X_te, Y_te), batch_size=64)
test_dl_noisy = make_loader(TensorDataset(X_te_noisy, Y_te_noisy), batch_size=64)
test_dl_randomized = make_loader(TensorDataset(X_te_randomized, Y_te_randomized), batch_size=64)
test_dls = [test_dl, test_dl_noisy, test_dl_randomized]
# X_te_flipped = torch.flip(X_te, [2, 3])
# flipped_test_dl = make_loader(TensorDataset(X_te_flipped, Y_te), batch_size=64)

Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified


In [8]:
def train_epoch(model, train_dl : DataLoader, opt, k = 50):
    ''' Trains model for one epoch on the provided dataloader, with optimizer opt. Logs stats every k batches.'''
    loss_func = nn.CrossEntropyLoss()
    model.train()
    model.cuda()

    netLoss = 0.0
    nCorrect = 0
    nTotal = 0
    for i, (xB, yB) in enumerate(tqdm(train_dl)):
        opt.zero_grad()
        xB, yB = xB.cuda(), yB.cuda()
        outputs = model(xB)
        loss = loss_func(outputs, yB)
        loss.backward()
        opt.step()
        netLoss += loss.item() * len(xB)  # why do we need to multiply by len(xB) ? 
        with torch.no_grad(): # we don't want to compute the gradients since it's just a callback
            _, preds = torch.max(outputs, dim=1)
            nCorrect += (preds == yB).float().sum()
            nTotal += preds.size(0)
        
        if (i+1) % k == 0:
            train_acc = nCorrect/nTotal
            avg_loss = netLoss/nTotal
            # print(f'\t [Batch {i+1} / {len(train_dl)}] Train Loss: {avg_loss:.3f} \t Train Acc: {train_acc:.3f}')
        torch.cuda.empty_cache()
          
    train_acc = nCorrect/nTotal
    avg_loss = netLoss/nTotal
    return avg_loss, train_acc


def evaluate(model, test_dl, loss_func=nn.CrossEntropyLoss().cuda()):
    ''' Returns loss, acc'''
    model.eval()
    model.cuda()
    nCorrect = 0.0
    nTotal = 0
    net_loss = 0.0
    with torch.no_grad():
        for (xb, yb) in test_dl:
            xb, yb = xb.cuda(), yb.cuda()
            outputs = model(xb)
            loss = len(xb) * loss_func(outputs, yb)
            _, preds = torch.max(outputs, dim=1)
            nCorrect += (preds == yb).float().sum()
            net_loss += loss.item()
            nTotal += preds.size(0)
    acc = nCorrect / float(nTotal)
    loss = net_loss / float(nTotal)
    return loss, acc

In [9]:
class Flatten(nn.Module):
    def forward(self, x): return x.view(x.size(0), x.size(1))

def make_cnn(c=64, num_classes=10, regularized=False):
    ''' Returns a 5-layer CNN with width parameter c. '''
    if regularized: 
        return nn.Sequential(
            # Layer 0
            nn.Conv2d(3, c, kernel_size=3, stride=1,
                      padding=1, bias=True),
            nn.BatchNorm2d(c),
            nn.Dropout(),
            nn.ReLU(),

            # Layer 1
            nn.Conv2d(c, c*2, kernel_size=3,
                      stride=1, padding=1, bias=True),
            nn.BatchNorm2d(c*2),
            nn.Dropout(),
            nn.ReLU(),
            nn.MaxPool2d(2),

            # Layer 2
            nn.Conv2d(c*2, c*4, kernel_size=3,
                      stride=1, padding=1, bias=True),
            nn.BatchNorm2d(c*4),
            nn.Dropout(),
            nn.ReLU(),
            nn.MaxPool2d(2),

            # Layer 3
            nn.Conv2d(c*4, c*8, kernel_size=3,
                      stride=1, padding=1, bias=True),
            nn.BatchNorm2d(c*8),
            nn.Dropout(),
            nn.ReLU(),
            nn.MaxPool2d(2),

            # Layer 4
            nn.MaxPool2d(4),
            Flatten(),
            nn.Linear(c*8, num_classes, bias=True)
        )
    else:
        return nn.Sequential(
            # Layer 0
            nn.Conv2d(3, c, kernel_size=3, stride=1,
                      padding=1, bias=True),
            nn.BatchNorm2d(c),
            nn.ReLU(),

            # Layer 1
            nn.Conv2d(c, c*2, kernel_size=3,
                      stride=1, padding=1, bias=True),
            nn.BatchNorm2d(c*2),
            nn.ReLU(),
            nn.MaxPool2d(2),

            # Layer 2
            nn.Conv2d(c*2, c*4, kernel_size=3,
                      stride=1, padding=1, bias=True),
            nn.BatchNorm2d(c*4),
            nn.ReLU(),
            nn.MaxPool2d(2),

            # Layer 3
            nn.Conv2d(c*4, c*8, kernel_size=3,
                      stride=1, padding=1, bias=True),
            nn.BatchNorm2d(c*8),
            nn.ReLU(),
            nn.MaxPool2d(2),

            # Layer 4
            nn.MaxPool2d(4),
            Flatten(),
            nn.Linear(c*8, num_classes, bias=True)
        )

## Under-Parametrized (c=4) 

In [10]:
model = make_cnn(c=4)
print('Number of trainable parameters of the current model ',  sum(p.numel() for p in model.parameters() if p.requires_grad))
opt = torch.optim.SGD(model.parameters(), lr=3e-3)  # from the LTH paper, where they trained Conv-4 architectures (c=4:5k epochs, c=64:15k epochs), with dropout
epochs = 5000
runnames = ['True-Labels', 'Noisy-Labels', 'Randomized Labels']
for i, (train_dl, test_dl) in enumerate(zip(train_dls, test_dls)):
    run = wandb.init(project="CS229br_hw0_memorizing_NN_Underparametrized", reinit=True)
    wandb.run.name = runnames[i]
    for i in range(epochs):
        print(f'Starting Epoch {i}')
        train_loss, train_acc = train_epoch(model, train_dl, opt)
        test_loss, test_acc = evaluate(model, test_dl)
        wandb.log({"epochs":i, "train_loss": train_loss, "train_accuracy": train_acc, "test_loss": test_loss, "test_accuracy": test_acc})
        print(f'Epoch {i}:\t Train Loss: {train_loss:.3f} \t Train Acc: {train_acc:.3f}\t Test Acc: {test_acc:.3f}')
    run.finish()

Number of trainable parameters of the current model  6666


epochs,28
train_loss,1.03369
train_accuracy,0.63634
test_loss,1.12723
test_accuracy,0.6028
_step,28
_runtime,170
_timestamp,1612451386


epochs,▁▁▁▂▂▂▃▃▃▃▃▄▄▄▅▅▅▅▅▆▆▆▇▇▇▇▇██
train_loss,█▆▅▅▄▄▄▃▃▃▃▃▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁
train_accuracy,▁▃▄▄▅▅▅▆▆▆▆▇▇▇▇▇▇▇▇▇█████████
test_loss,█▆▆▅▅▄▄▄▄▃▃▃▃▃▂▃▃▂▂▂▂▁▁▃▂▂▂▁▁
test_accuracy,▁▂▃▄▄▄▅▅▅▆▆▆▆▆▇▆▆▇▇▇▇██▇▇▇▇██
_step,▁▁▁▂▂▂▃▃▃▃▃▄▄▄▅▅▅▅▅▆▆▆▇▇▇▇▇██
_runtime,▁▁▂▂▂▂▂▃▃▃▄▄▄▄▅▅▅▅▅▆▆▆▆▇▇▇▇██
_timestamp,▁▁▂▂▂▂▂▃▃▃▄▄▄▄▅▅▅▅▅▆▆▆▆▇▇▇▇██


Starting Epoch 0


  0%|          | 0/782 [00:00<?, ?it/s]

...

Epoch 54:	 Train Loss: 0.887 	 Train Acc: 0.688	 Test Acc: 0.639
Starting Epoch 55


  0%|          | 0/782 [00:00<?, ?it/s]

Epoch 55:	 Train Loss: 0.884 	 Train Acc: 0.690	 Test Acc: 0.610
Starting Epoch 56


  0%|          | 0/782 [00:00<?, ?it/s]

RuntimeError: DataLoader worker (pid 30808) is killed by signal: Killed. 